In [1]:
import openai
from openai import OpenAI
import re
from pykospacing import Spacing
from hanspell import spell_checker
import pandas as pd
from datetime import datetime, timedelta
# .env파일에서 OPENAI_API_KEY 가져오기
import os
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'pykospacing'

In [2]:
load_dotenv()  # 이 코드를 추가하여 .env 파일을 명시적으로 로드합니다.
api_key = os.getenv('OPENAI_API_KEY')
# openai api_key 설정
openai.api_key = api_key
client = OpenAI(api_key=api_key) # your openai api key

# print("api_key:", api_key)

In [3]:
# 특수 문자 제거 함수
def clean_text(text):
    # 줄바꿈 문자 포함하여 특수 문자 제거
    text = re.sub(r'[^ㄱ-ㅎ가-힣a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()  # 공백 문자를 하나의 공백으로 치환하고 양쪽 공백 제거
    return text

In [16]:
# 한국어 띄어쓰기가 되어있지 않은 문장을 띄어쓰기를 한 문장으로 변환하는 함수
def kospacing_text(text):
    spacing = Spacing()
    return spacing(text)



In [21]:
# 한국어 네이버 한글 맞춤법 검사기를 이용하여 맞춤법을 검사하고, 검사 결과를 반환하는 함수
def spell_check(text):
    try:
        spelled_sent = spell_checker.check(text)
        print(spelled_sent)
        hanspell_sent = spelled_sent.checked
        print(hanspell_sent)
        return hanspell_sent
    except KeyError as e:
        print(f"KeyError: {e} - API 응답에서 예상 키를 찾을 수 없습니다.")
        return text  # 키 오류가 발생하면 원본 텍스트를 반환


In [23]:
print(spell_check("맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "))



KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 


In [6]:
# 언어 감지 함수
def detect_language(text):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a language detection assistant."},
            {"role": "user", "content": f"다음 텍스트의 주요 언어를 감지해 주세요: {text}"}
        ],
        max_tokens=5
    )
    print(response)
    return response.choices[0].message.content.strip()

In [7]:
# 텍스트 임베딩 생성 함수
def get_chatgpt_embedding(text):
    response = client.embeddings.create(
        input = text, 
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [8]:
# ISO 8601 형식의 duration을 초 단위로 변환하는 함수
def parse_duration(duration):
    duration = duration.replace('PT', '')
    hours = minutes = seconds = 0
    if 'H' in duration:
        hours, duration = duration.split('H')
        hours = int(hours)
    if 'M' in duration:
        minutes, duration = duration.split('M')
        minutes = int(minutes)
    if 'S' in duration:
        seconds = int(duration.rstrip('S'))
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return total_seconds

In [9]:
# 데이터 전처리를 위한 데이터 적재
data = pd.read_csv('./youtube_channel_comments_data_20240606_104600.csv')


In [10]:
# 데이터 분석을 위해 날짜 데이터 열(channelPublishedAt, videoPublishedAt, commentPublishedAt)을 
# 오브젝트타입에서  datetime으로 변경
data['channelPublishedAt'] = pd.to_datetime(data['channelPublishedAt'], format='mixed')
data['videoPublishedAt'] = pd.to_datetime(data['videoPublishedAt'], format='mixed')
data['commentPublishedAt'] = pd.to_datetime(data['commentPublishedAt'], format='mixed')

In [11]:
# 비디오(영상)별 재생시간을 나타내는 duration 컬럼에 대한 전처리
# duration 컬럼의 데이터를 초 단위로 변환
data['duration'] = data['duration'].apply(parse_duration)


In [12]:
# 누락된 데이터 처리
data.fillna({'commentText': 'No comment'}, inplace=True)

In [13]:
# 데이터 타입 검증 및 변환
data['subscriberCount'] = data['subscriberCount'].astype(int)
data['viewCount'] = data['viewCount'].astype(int)
data['likeCount'] = data['likeCount'].astype(int)
data['dislikeCount'] = data['dislikeCount'].astype(int)
data['commentLikeCount'] = data['commentLikeCount'].astype(int)

In [14]:
# 댓글, 답글 데이터 전처리
data['cleaned_commentText'] = data['commentText'].apply(clean_text)
# data['language'] = data['cleaned_commentText'].apply(detect_language)

# 텍스트 임베딩 생성
# data['embedding'] = data['cleaned_commentText'].apply(get_chatgpt_embedding)

In [17]:
# 한국어 띄어쓰기 전처리
data['cleaned_commentText'] = data['cleaned_commentText'].apply(kospacing_text)


In [22]:
# 한국어 맞춤법 검사
# data['cleaned_commentText'] = data['cleaned_commentText'].apply(spell_check)

KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 'result' - API 응답에서 예상 키를 찾을 수 없습니다.
KeyError: 're

KeyboardInterrupt: 

In [24]:
# 영상 제목 데이터 전처리
data['videoTitle'] = data['videoTitle'].apply(clean_text)


In [25]:
# 리스트를 문자열로 변환
data = data.applymap(lambda x: str(x) if isinstance(x, list) else x)

# 중복 데이터 제거
data.drop_duplicates(inplace=True)

/tmp/ipykernel_1659/521839354.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: str(x) if isinstance(x, list) else x)


In [26]:
# 카테고리 데이터 처리
data['channelId'] = data['channelId'].astype('category')
data['videoId'] = data['videoId'].astype('category')
data['commentId'] = data['commentId'].astype('category')

In [27]:
# 결과 저장
data.to_csv('./youtube_channel_comments_cleaneddata.csv', index=False)

# 데이터 확인
print(data.head())

                  channelId    channelTitle        channelPublishedAt  \
0  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기 2017-03-02 02:28:38+00:00   
1  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기 2017-03-02 02:28:38+00:00   
2  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기 2017-03-02 02:28:38+00:00   
3  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기 2017-03-02 02:28:38+00:00   
4  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기 2017-03-02 02:28:38+00:00   

   subscriberCount      videoId                        videoTitle  \
0          1780000  qlLkiUU0o1A  우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호   
1          1780000  qlLkiUU0o1A  우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호   
2          1780000  qlLkiUU0o1A  우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호   
3          1780000  qlLkiUU0o1A  우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호   
4          1780000  qlLkiUU0o1A  우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호   

              videoAuthorId          videoPublishedAt  duration  viewCount  \
0  UCMFk5S7g5DY-CZNVh_Kyz_A 2024-06-05 08:18:39+00:00        56     